# Importing Repositories

In [45]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import holidays
warnings.filterwarnings("ignore")
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize']=(20,10)

# Functions

In [46]:
def missing_zero_values_table(df): 
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Our selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
    
        return mz_table

# Importing data files

## Paths for files

In [47]:
path_price = r'C:\Users\2836589S\OneDrive - University of Glasgow\Data\DayAheadPrices_12.1.D'
path_totalload = r'C:\Users\2836589S\OneDrive - University of Glasgow\Data\DayAheadTotalLoadForecast'
path_windsolar = r'C:\Users\2836589S\OneDrive - University of Glasgow\Data\DayAheadGenerationForecastForWindAndSolar_14.1.D'

## Price Timeseries Importing

In [48]:

df_price = pd.read_csv(r'C:\Users\2836589S\OneDrive - University of Glasgow\Data\Data Frames\price_timeseries_outliers.csv', 
parse_dates=['DateTime'])


In [49]:
df_price = df_price[['DateTime', 'Year', 'Month','Week', 'Day', 'Hour', 'Price', 'Outlier']]
df_price

,DateTime,Year,Month,Week,Day,Hour,Price,Outlier
0,2015-01-01 00:00:00,2015,1,1,3,0,0.00,N
1,2015-01-01 01:00:00,2015,1,1,3,1,0.00,N
2,2015-01-01 02:00:00,2015,1,1,3,2,0.00,N
3,2015-01-01 03:00:00,2015,1,1,3,3,0.00,N
4,2015-01-01 04:00:00,2015,1,1,3,4,0.00,N
...,...,...,...,...,...,...,...,...
52529,2020-12-31 18:00:00,2020,12,53,3,18,71.30,N
52530,2020-12-31 19:00:00,2020,12,53,3,19,61.04,N
52531,2020-12-31 20:00:00,2020,12,53,3,20,60.39,N
52532,2020-12-31 21:00:00,2020,12,53,3,21,57.96,N


## Wind and Solar Timeseries Importing

In [50]:
f_windsolar = glob.glob(path_windsolar + "/*.csv")
dwindsolar = [pd.read_csv(f, sep='\t', parse_dates=['DateTime']) for f in f_windsolar]

df_windsolar = pd.concat(dwindsolar)
df_windsolar = df_windsolar[df_windsolar['AreaCode'] == '10YGB----------A']
df_windsolar = df_windsolar[['DateTime', 'ProductionType', 'AggregatedGenerationForecast']]
df_windsolar.sort_values(by='DateTime', ascending=True)
df_windsolar.reset_index(drop=True, inplace=True)

In [51]:
df_windsolar

,DateTime,ProductionType,AggregatedGenerationForecast
0,2014-12-30 00:00:00,Solar,0.00
1,2014-12-30 01:00:00,Solar,0.00
2,2014-12-30 02:00:00,Solar,0.00
3,2014-12-30 03:00:00,Solar,0.00
4,2014-12-30 04:00:00,Solar,0.00
...,...,...,...
509107,2021-06-14 19:00:00,Wind Offshore,1563.23
509108,2021-06-14 20:00:00,Wind Offshore,1590.47
509109,2021-06-14 21:00:00,Wind Offshore,1574.14
509110,2021-06-14 22:00:00,Wind Offshore,1450.72


### Splitting the WindSolar Dataset into 3 Datasets per Solar, Wind Offshore, Wind Onshore

In [52]:
df_windsolar['ProductionType'].unique()

array(['Solar', 'Wind Offshore', 'Wind Onshore'], dtype=object)

In [53]:
df_solar = df_windsolar[df_windsolar['ProductionType'] == 'Solar']
df_solar.sort_values(by='DateTime', ascending=True)
df_solar.reset_index(drop=True, inplace=True)
df_solar.rename(columns={'AggregatedGenerationForecast':'SolarGeneration'}, inplace=True)
df_solar = df_solar[['DateTime','SolarGeneration']]
df_solar

,DateTime,SolarGeneration
0,2014-12-30 00:00:00,0.0
1,2014-12-30 01:00:00,0.0
2,2014-12-30 02:00:00,0.0
3,2014-12-30 03:00:00,0.0
4,2014-12-30 04:00:00,0.0
...,...,...
169699,2021-06-14 19:00:00,227.0
169700,2021-06-14 20:00:00,11.5
169701,2021-06-14 21:00:00,0.0
169702,2021-06-14 22:00:00,0.0


In [54]:
# Taking of the outliers
q1 , q3 = np.percentile(df_solar['SolarGeneration'], [25,75])
iqr = q3 - q1
ul = q3 + 1.5*iqr
ll = q1 - 1.5*iqr
df_solar = df_solar[(df_solar['SolarGeneration'] > ll) | (df_solar['SolarGeneration'] < ul) ]

In [55]:
df_wind_off = df_windsolar[df_windsolar['ProductionType'] == 'Wind Offshore']
df_wind_off.sort_values(by='DateTime', ascending=True)
df_wind_off.reset_index(drop=True, inplace=True)
df_wind_off.rename(columns={'AggregatedGenerationForecast':'WindOffGeneration'}, inplace=True)
df_wind_off = df_wind_off[['DateTime', 'WindOffGeneration']]
df_wind_off

,DateTime,WindOffGeneration
0,2014-12-30 00:00:00,996.58
1,2014-12-30 01:00:00,1059.20
2,2014-12-30 02:00:00,1123.27
3,2014-12-30 03:00:00,1177.54
4,2014-12-30 04:00:00,1188.21
...,...,...
169699,2021-06-14 19:00:00,1563.23
169700,2021-06-14 20:00:00,1590.47
169701,2021-06-14 21:00:00,1574.14
169702,2021-06-14 22:00:00,1450.72


In [56]:
# Taking care of the outliers
q1 , q3 = np.percentile(df_wind_off['WindOffGeneration'], [25,75])
iqr = q3 - q1
ul = q3 + 1.5*iqr
ll = q1 - 1.5*iqr
df_wind_off = df_wind_off[(df_wind_off['WindOffGeneration'] > ll) | (df_wind_off['WindOffGeneration']) ]

In [57]:
df_wind_on = df_windsolar[df_windsolar['ProductionType'] == 'Wind Onshore']
df_wind_on.sort_values(by='DateTime', ascending=True)
df_wind_on.reset_index(drop=True, inplace=True)
df_wind_on.rename(columns={'AggregatedGenerationForecast':'WindOnGeneration'}, inplace=True)
df_wind_on = df_wind_on[['DateTime','WindOnGeneration']]
df_wind_on

,DateTime,WindOnGeneration
0,2014-12-30 00:00:00,2363.80
1,2014-12-30 01:00:00,2413.88
2,2014-12-30 02:00:00,2450.75
3,2014-12-30 03:00:00,2500.78
4,2014-12-30 04:00:00,2536.07
...,...,...
169699,2021-06-14 19:00:00,4896.71
169700,2021-06-14 20:00:00,3928.19
169701,2021-06-14 21:00:00,3277.03
169702,2021-06-14 22:00:00,2880.91


In [58]:
# Taking care of the outliers
q1 , q3 = np.percentile(df_wind_on['WindOnGeneration'], [25,75])
iqr = q3 - q1
ul = q3 + 1.5*iqr
ll = q1 - 1.5*iqr
df_wind_on = df_wind_on[(df_wind_on['WindOnGeneration'] > ll) | (df_wind_on['WindOnGeneration']) ]

## Total Load Timeseries Importing

In [59]:
f_totalload = glob.glob(path_totalload + "/*.csv")
dtotalload = [pd.read_csv(f, encoding='utf-16', sep='\t', parse_dates=['DateTime']) for f in f_totalload]

df_totalload = pd.concat(dtotalload)
df_totalload = df_totalload[df_totalload['AreaCode'] == '10YGB----------A']
df_totalload = df_totalload[['DateTime', 'TotalLoadValue']]
df_totalload.reset_index(drop=True, inplace=True)


In [60]:
df_totalload.sort_values(by=['DateTime'], ascending=True)
df_totalload.reset_index(drop=True, inplace=True)
df_totalload

,DateTime,TotalLoadValue
0,2014-12-29 00:00:00,28798.0
1,2014-12-29 02:00:00,29534.0
2,2014-12-29 00:30:00,29961.0
3,2014-12-29 02:30:00,29627.0
4,2014-12-29 04:30:00,26717.0
...,...,...
202693,2020-09-30 14:30:00,34726.0
202694,2020-09-30 16:30:00,36430.0
202695,2020-09-30 18:30:00,36366.0
202696,2020-09-30 20:30:00,29449.0


# Merging Datasets

In [61]:
df_price = df_price[df_price['Outlier'] == 'N']
df_merged = pd.merge(df_price, df_solar, how='left', on='DateTime')
df_merged = pd.merge(df_merged, df_wind_off, how='left', on='DateTime')
df_merged = pd.merge(df_merged, df_wind_on, how='left', on='DateTime')

In [62]:
df_merged.drop_duplicates(subset=['DateTime'], keep='first', inplace=True)

In [63]:
df_model = pd.DataFrame()
df_model = df_merged[['Price','Year', 'Month', 'Day', 'Hour', 'SolarGeneration', 'WindOffGeneration',
'WindOnGeneration']]
df_model.set_index(df_merged['DateTime'], inplace=True)
df_model

,Price,Year,Month,Day,Hour,SolarGeneration,WindOffGeneration,WindOnGeneration
DateTime,,,,,,,,
2015-01-01 00:00:00,0.00,2015,1,3,0,0.0,2987.05,4013.58
2015-01-01 01:00:00,0.00,2015,1,3,1,0.0,2926.71,4074.53
2015-01-01 02:00:00,0.00,2015,1,3,2,0.0,2867.00,4157.48
2015-01-01 03:00:00,0.00,2015,1,3,3,0.0,2808.72,4229.50
2015-01-01 04:00:00,0.00,2015,1,3,4,0.0,2769.59,4133.56
...,...,...,...,...,...,...,...,...
2020-12-31 18:00:00,71.30,2020,12,3,18,0.0,2644.97,3028.41
2020-12-31 19:00:00,61.04,2020,12,3,19,0.0,2707.55,3046.72
2020-12-31 20:00:00,60.39,2020,12,3,20,0.0,2695.01,3053.60


## Transforming Solar, Wind Onshore and Wind Offshore Generation values

In [64]:
df_model['SolarGeneration'] = df_model['SolarGeneration'].transform('sqrt')
df_model['WindOffGeneration'] = df_model['WindOffGeneration'].transform('sqrt')
df_model['WindOnGeneration'] = df_model['WindOnGeneration'].transform('sqrt')

# Lasso Regression

In [65]:
lags = list(range(1,168))


In [66]:
def lag_col(df, variable, lags):
    for lag in lags:
        df[str(variable) + '_' + str(lag)] = df[variable].shift(lag)
    return df
    

In [67]:
lag_col(df_model, 'Price', lags)
lag_col(df_model, 'SolarGeneration', lags)
lag_col(df_model, 'WindOffGeneration', lags)
lag_col(df_model, 'WindOnGeneration', lags)

,Price,Year,Month,Day,Hour,SolarGeneration,WindOffGeneration,WindOnGeneration,Price_1,Price_2,...,WindOnGeneration_158,WindOnGeneration_159,WindOnGeneration_160,WindOnGeneration_161,WindOnGeneration_162,WindOnGeneration_163,WindOnGeneration_164,WindOnGeneration_165,WindOnGeneration_166,WindOnGeneration_167
DateTime,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.00,2015,1,3,0,0.0,54.653911,63.352822,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00,0.00,2015,1,3,1,0.0,54.099076,63.832045,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 02:00:00,0.00,2015,1,3,2,0.0,53.544374,64.478524,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 03:00:00,0.00,2015,1,3,3,0.0,52.997358,65.034606,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 04:00:00,0.00,2015,1,3,4,0.0,52.626894,64.292768,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 18:00:00,71.30,2020,12,3,18,0.0,51.429272,55.030991,76.00,57.54,...,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478,67.321022,68.768888
2020-12-31 19:00:00,61.04,2020,12,3,19,0.0,52.034123,55.197101,71.30,76.00,...,46.562109,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478,67.321022
2020-12-31 20:00:00,60.39,2020,12,3,20,0.0,51.913486,55.259388,61.04,71.30,...,44.703132,46.562109,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478


In [68]:
df_model.drop(columns=['Year','Month','Day','Hour'], inplace=True)

In [69]:
df_model.dropna(inplace=True)

In [70]:
df_model

,Price,SolarGeneration,WindOffGeneration,WindOnGeneration,Price_1,Price_2,Price_3,Price_4,Price_5,Price_6,...,WindOnGeneration_158,WindOnGeneration_159,WindOnGeneration_160,WindOnGeneration_161,WindOnGeneration_162,WindOnGeneration_163,WindOnGeneration_164,WindOnGeneration_165,WindOnGeneration_166,WindOnGeneration_167
DateTime,,,,,,,,,,,,,,,,,,,,,
2015-01-08 12:00:00,46.05,23.769729,48.260336,65.519386,47.18,41.98,48.10,37.31,36.79,37.91,...,65.442188,64.310108,63.262153,62.639764,63.287361,64.292768,65.034606,64.478524,63.832045,63.352822
2015-01-08 13:00:00,41.52,23.388031,49.602218,63.875191,46.05,47.18,41.98,48.10,37.31,36.79,...,68.319617,65.442188,64.310108,63.262153,62.639764,63.287361,64.292768,65.034606,64.478524,63.832045
2015-01-08 14:00:00,35.55,22.978251,49.935158,62.397997,41.52,46.05,47.18,41.98,48.10,37.31,...,69.889699,68.319617,65.442188,64.310108,63.262153,62.639764,63.287361,64.292768,65.034606,64.478524
2015-01-08 15:00:00,35.10,22.583180,50.113072,61.135260,35.55,41.52,46.05,47.18,41.98,48.10,...,70.940398,69.889699,68.319617,65.442188,64.310108,63.262153,62.639764,63.287361,64.292768,65.034606
2015-01-08 16:00:00,49.96,16.248077,50.086425,59.869191,35.10,35.55,41.52,46.05,47.18,41.98,...,71.690306,70.940398,69.889699,68.319617,65.442188,64.310108,63.262153,62.639764,63.287361,64.292768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 18:00:00,71.30,0.000000,51.429272,55.030991,76.00,57.54,62.09,65.60,64.90,63.01,...,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478,67.321022,68.768888
2020-12-31 19:00:00,61.04,0.000000,52.034123,55.197101,71.30,76.00,57.54,62.09,65.60,64.90,...,46.562109,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478,67.321022
2020-12-31 20:00:00,60.39,0.000000,51.913486,55.259388,61.04,71.30,76.00,57.54,62.09,65.60,...,44.703132,46.562109,47.850183,49.701610,52.908506,55.829383,58.929449,61.072089,63.249111,65.433478


## Scikit Learn approach with LassoCV

In [71]:
X = df_model.drop(columns='Price')
y = df_model.Price

In [72]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

In [73]:
# Taken from Sklearn.model_selection documentation

for train, test in tscv.split(X):
    print("%s %s" % (train, test))

[   0    1    2 ... 8269 8270 8271] [ 8272  8273  8274 ... 16541 16542 16543]
[    0     1     2 ... 16541 16542 16543] [16544 16545 16546 ... 24813 24814 24815]
[    0     1     2 ... 24813 24814 24815] [24816 24817 24818 ... 33085 33086 33087]
[    0     1     2 ... 33085 33086 33087] [33088 33089 33090 ... 41357 41358 41359]
[    0     1     2 ... 41357 41358 41359] [41360 41361 41362 ... 49629 49630 49631]


In [74]:
from sklearn.linear_model import LassoCV

reg = LassoCV(cv=tscv, random_state=0).fit(X,y)

In [75]:
# R squared

reg.score(X,y)

0.8620085143362572

In [76]:
# Alpha value of our regressor

reg.alpha_

0.15394068411808812

In [77]:
# Matrix of alpha values used during the fitting


reg.alphas_

array([153.94068412, 143.56559723, 133.88975647, 124.866035  ,
       116.45048215, 108.6021094 , 101.28269071,  94.45657634,
        88.09051923,  82.15351306,  76.61664124,  71.45293604,
        66.6372473 ,  62.14611986,  57.95767938,  54.05152576,
        50.40863383,  47.01126062,  43.842859  ,  40.88799704,
        38.13228289,  35.56229464,  33.16551501,  30.9302703 ,
        28.8456736 ,  26.90157174,  25.08849584,  23.39761519,
        21.82069424,  20.35005248,  18.97852705,  17.69943784,
        16.50655497,  15.39406841,  14.35655972,  13.38897565,
        12.4866035 ,  11.64504822,  10.86021094,  10.12826907,
         9.44565763,   8.80905192,   8.21535131,   7.66166412,
         7.1452936 ,   6.66372473,   6.21461199,   5.79576794,
         5.40515258,   5.04086338,   4.70112606,   4.3842859 ,
         4.0887997 ,   3.81322829,   3.55622946,   3.3165515 ,
         3.09302703,   2.88456736,   2.69015717,   2.50884958,
         2.33976152,   2.18206942,   2.03500525,   1.89

In [78]:
predictors = X.columns.to_list()

coef_table = pd.DataFrame(list(predictors)).copy()
coef_table.insert(len(coef_table.columns), "Coefficients", reg.coef_.transpose())
coef_table.sort_values(by='Coefficients', ascending=False).reset_index(drop=True).head(20)

,0,Coefficients
0,Price_1,0.792704
1,Price_24,0.111224
2,Price_22,0.093994
3,SolarGeneration_1,0.074918
4,Price_23,0.067449
5,SolarGeneration_23,0.043188
6,WindOnGeneration_3,0.039807
7,Price_48,0.036692
8,Price_21,0.035060
9,SolarGeneration_6,0.028103


## Pycaret

In [79]:
from pycaret.regression import *

In [80]:
s = setup(data=df_model, target='Price', fold_strategy=tscv, fold=5)


,Description,Value
0,Session id,4563
1,Target,Price
2,Target type,Regression
3,Data shape,"(49632, 672)"
4,Train data shape,"(34742, 672)"
5,Test data shape,"(14890, 672)"
6,Numeric features,671
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [81]:
lasso = create_model('lasso')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.5467,24.7472,4.9747,0.8560,0.1759,0.4464
1,3.4367,24.0047,4.8995,0.8626,0.1672,0.1433
2,3.4344,23.3809,4.8354,0.8642,0.1773,0.4280
3,3.5473,24.1008,4.9093,0.8553,0.1572,0.2031
4,3.5381,24.8043,4.9804,0.8550,0.1694,0.4757
Mean,3.5006,24.2076,4.9198,0.8586,0.1694,0.3393
Std,0.0532,0.5259,0.0535,0.0039,0.0072,0.1378


In [82]:
evaluate_model(lasso)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [83]:
df_model['WindOffGeneration'].describe()

count    49632.000000
mean        46.725891
std         19.710505
min          7.041307
25%         31.302835
50%         45.759371
75%         60.025599
max         99.060588
Name: WindOffGeneration, dtype: float64